In [1]:
import os
import json
import pandas as pd
import traceback

In [ ]:
pip install --upgrade langchain

In [9]:
from langchain_openai import ChatOpenAI

In [11]:
from dotenv import load_dotenv

load_dotenv() # take environment variables from .env

True

In [12]:
KEY = os.getenv("OPENAI_API_KEY")

In [14]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [17]:
pip install langchain langchain-community

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [32]:
RESPONSE_JSON = {
    "1": {
      "mcq": "multiple choice question",
      "options": {
          "a": "choice here",
          "b": "choice here",
          "c": "choice here",
          "d": "choice here"
      },
      "correct": "correct answer",
    },
    "2": {
      "mcq": "multiple choice question",
      "options": {
          "a": "choice here",
          "b": "choice here",
          "c": "choice here",
          "d": "choice here"
      },
      "correct": "correct answer",
    },
    "3": {
      "mcq": "multiple choice question",
      "options": {
          "a": "choice here",
          "b": "choice here",
          "c": "choice here",
          "d": "choice here"
      },
      "correct": "correct answer",
    }
}

In [19]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [22]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [25]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [26]:
TEMPLATE2 = """
You are an expert english gramarian and writter. Given a Multiple Choice Quiz for {subject} students. \
You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity,
if the quiz is not at per with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student
Quiz_MCQs:
{quiz}

Check from an export English Writer of the above quiz:
""" 

In [27]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [28]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [29]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [30]:
file_path = r"C:\Users\LENOVO\OneDrive\Desktop\Project\GenAI\mcqgen\data.txt"

In [31]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [33]:
# Serialize the Python dictionary into JSON-Formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [34]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [35]:
# https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

# How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text" : TEXT,
            "number" : NUMBER,
            "subject" : SUBJECT,
            "tone" : TONE,
            "response_json" : json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6724\248961861.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [36]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2535
Prompt Tokens:1788
Completion Tokens:747
Total Cost:0.0020145


In [37]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [41]:
quiz = response.get("quiz")

In [43]:
quiz = json.loads(quiz)

In [44]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [45]:
quiz_table_data

[{'MCQ': 'Who coined the term machine learning in 1959?',
  'Choices': 'a : Donald Hebb | b : Tom M. Mitchell | c : Arthur Samuel | d : Walter Pitts',
  'Correct': 'c'},
 {'MCQ': 'Which book introduced a theoretical neural structure formed by certain interactions among nerve cells?',
  'Choices': 'a : The Organization of Behavior | b : Learning Machines | c : Computing Machinery and Intelligence | d : The History of Machine Learning',
  'Correct': 'a'},
 {'MCQ': 'What was the name of the experimental learning machine developed by Raytheon Company in the early 1960s?',
  'Choices': 'a : Cybertron | b : HAL 9000 | c : Skynet | d : R2-D2',
  'Correct': 'a'},
 {'MCQ': 'Who provided a formal definition of the algorithms studied in the machine learning field?',
  'Choices': 'a : Donald Hebb | b : Tom M. Mitchell | c : Arthur Samuel | d : Walter Pitts',
  'Correct': 'b'},
 {'MCQ': 'What are the two objectives of modern-day machine learning?',
  'Choices': 'a : To confuse data and make random 

In [46]:
quiz = pd.DataFrame(quiz_table_data)

In [47]:
quiz.to_csv("machinelearning.csv", index=False)